In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import os
from os import listdir
from os.path import isfile, join

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
# parse Cell Volumes (manual segmentation)

CV_dir_1 = "/Volumes/TOB_WD2/Microscopy/20230411_Centrosome_Cenp-j/Cell_Volumes"
CV_dir_2 = "/Volumes/TOB_WD2/Microscopy/20230504_Centrosome_Cenp-j/Cell_Volumes"
CV_dir_3 = "/Volumes/TOB_WD2/Microscopy/20230626_Centrosome_Cenp-j/Cell_Volumes"
CV_dir_4 = "/Volumes/TOB_WD2/Microscopy/20230720_Centrosome_Cenp-j/Cell_Volumes"

def concat_CV(folder):
    CV_dataframes = []
    for root, dirs, files in os.walk(folder):
        for file in files:
            if not file.startswith("."):
                filepath = root + os.sep + file
                experiment = root.split("/")[4][:8]
                if filepath.endswith(".csv"):
                    single_df = pd.read_csv(filepath)
                    single_df = single_df.rename(
                            columns = {
                                "Volume": "Cell_Volume_um3",
                                "SurfaceArea": "Cell_Surface_Area_um2"
                            }
                    )
                    single_df = single_df[["Cell_ID", "Cell_Volume_um3", "Cell_Surface_Area_um2"]]
                    single_df['Experiment'] = experiment

                    CV_dataframes.append(single_df)
                else:
                    pass
            else:
                pass
    CV_df = pd.concat(CV_dataframes)
    return CV_df 

CV_df_1 = concat_CV(CV_dir_1)
CV_df_2 = concat_CV(CV_dir_2)
CV_df_3 = concat_CV(CV_dir_3)
CV_df_4 = concat_CV(CV_dir_4)

CV_df = pd.concat([CV_df_1, CV_df_2, CV_df_3, CV_df_4])

print(CV_df.shape)
CV_df.head(1)

(715, 4)


,Cell_ID,Cell_Volume_um3,Cell_Surface_Area_um2,Experiment
0,20230411_R1E309_Cenp-J_100X_SirActinSirDNA_Full+CCB1uM_001-1,3274.698,1108.628,20230411


In [3]:
# parse Spindle3D morphometry
# parse Plugin Measurements

input_dir1 = "/Volumes/TOB_WD2/Microscopy/20230411_Centrosome_Cenp-j/Spindle3D/Correct"
input_dir2 = "/Volumes/TOB_WD2/Microscopy/20230504_Centrosome_Cenp-j/Spindle3D/Correct"
input_dir3 = "/Volumes/TOB_WD2/Microscopy/20230626_Centrosome_Cenp-j/Spindle3D/Correct"
input_dir4 = "/Volumes/TOB_WD2/Microscopy/20230720_Centrosome_Cenp-j/Spindle3D/Correct"

spindle_input_folders = [
    input_dir1, input_dir2, input_dir3, input_dir4
]

def concat_spindle3d(folderlist):
    dataframes = []
    for folder in folderlist:    
        for root, dirs, files in os.walk(folder):
            for file in files:
                filepath = root + os.sep + file  
                if filepath.endswith(".txt"):
                    single_df = pd.read_csv(filepath, sep = " ", delimiter = "\t", encoding = 'unicode_escape')
                    dataframes.append(single_df)  
                else:
                    pass
    df = pd.concat(dataframes)
    return df

spindle_df = concat_spindle3d(spindle_input_folders)

print("Finished parsing data.")

def get_nakedname(path):
    # parse cell ids from file name in spindle3D table
    base = os.path.basename(str(path))
    naked = os.path.splitext(base)[0]
    return naked

# modify spindle 3d dataframe
spindle_df["Cell_ID"] = spindle_df.Path_InputImage.apply(get_nakedname)

subselection_spindle = [
    "Cell_ID",
    "Spindle_Angle_Degrees",  
    "Spindle_Length_um", 
    "Spindle_Volume_um3", 
    "Spindle_Width_Avg_um", 
    "Tubulin_Spindle_Average_Intensity",
    "Chromatin_Volume_um3",
    "MetaphasePlate_Length_um",
    "MetaphasePlate_Width_um",
    "Version"
]
spindle_df = spindle_df[subselection_spindle]
print(spindle_df.shape)
spindle_df.head(1)

Finished parsing data.
(589, 10)


,Cell_ID,Spindle_Angle_Degrees,Spindle_Length_um,Spindle_Volume_um3,Spindle_Width_Avg_um,Tubulin_Spindle_Average_Intensity,Chromatin_Volume_um3,MetaphasePlate_Length_um,MetaphasePlate_Width_um,Version
0,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = spindle_df.merge(CV_df)
df.shape

(580, 13)

In [5]:
df["Condition"] = df.Cell_ID.str.split("_").str.get(5)
df["Spindle_Occupancy"] = df.Spindle_Volume_um3 / df.Cell_Volume_um3 * 100

In [6]:
def rename_condition(x):
    if x == "Full+DMSO":
        return "1_DMSO"
    elif x == "Full+CCB1uM":
        return "2_1µM"
    elif x == "Full+CCB2uM":
        return "3_2µM"
    elif x == "Full+CCB5uM":
        return "4_5µM"
    elif x == "Full+CCB10uM":
        return "5_10µM"
    elif x == "Full+N2B27+RA":
        return "6_N2B27+RA"
    elif x == "N2B27+RA":
        return "6_N2B27+RA"
    else:
        pass
df["Condition"] = df.Condition.apply(rename_condition)

In [7]:
# Export Dataframe

destination = "/Volumes/TOB_WD2/Image_Analysis/CCB02/Dataframes" + "/MasterDataFrame_CCB02_Live.csv"
df.to_csv(destination)
print("Successfully saved dataframe at {}".format(destination))

Successfully saved dataframe at /Volumes/TOB_WD2/Image_Analysis/CCB02/Dataframes/MasterDataFrame_CCB02_Live.csv
